In [53]:
import os
# os.chdir('/home/jovyan/scrgt-bdso-cc-datascience/datapipeline'
os.chdir('/Users/rayduong/Documents/GitHub/scrgt-bdso-cc-datascience/datapipeline')
ESPORT= os.environ.get('ESPORT')
ESHOST= os.environ.get('ESHOST')



## Imports:

In [54]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing
import datetime
import pickle
import json

from IPython.display import HTML
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

## Create sentence embeddings from Elmo via TensorFlow Hub
https://tfhub.dev/google/elmo/1

In [55]:
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

## Read Plot Sentences

In [56]:
# Change Directory to Data
# read from data
# Create elmoWeights Folder, in not exists
# Write weights to data/emloWeights


# datadir = os.path.join(rootdir, 'data') 
path_to_json = './data/person/themoviedb/'

elmodir = os.getcwd()+'/models/'

# create elmoWeights directory - if it does not exist
if not os.path.exists(elmodir):
    os.makedirs(elmodir) 

## READ PLOT DATA - ALL from PKL file

In [57]:

json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and os.stat(path_to_json+pos_json).st_size>0]
len(json_files)


bio_dict = {}
for f in json_files:
    data = json.load(open(f) )
    bio = data["biography"]
    bio_dict[data["_actorID"]] = bio
    


In [58]:
sentencesAll = list(bio_dict.values())
sentencesAll[0]

"An American actor and former professional wrestler, mixed martial artist, and bodybuilder.\n\nFrom 2000–2010 and 2013–2014, he was signed to WWE under the ring name Batista, where he became a six-time world champion, winning WWE's World Heavyweight Championship four times and the WWE Championship twice. He holds the record for the longest reign as World Heavyweight Champion at 282 days, and has also won the World Tag Team Championship three times (twice with Ric Flair and once with John Cena) and the WWE Tag Team Championship once (with Rey Mysterio). He was the winner of the 2005 Royal Rumble match and went on to headline WrestleMania 21, one of the top five highest-grossing pay-per-view events in professional wrestling history. When Bautista returned to WWE in 2014, he won the 2014 Royal Rumble match, after which he headlined WrestleMania XXX.\n\nIn August 2012, Bautista signed a contract with Classic Entertainment & Sports to fight in mixed martial arts. He won his MMA debut on Oct

In [59]:
def cleanSentences(sentences):
    punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'  #TBD
        #TBD - find sentence splitter - 
        # remove punctuation, strip white space; remove non-standard characters; remove numbers; 
    i = 0
    for s in sentences:
        s = s.strip()
        s = s.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ').replace("[0-9]", " ")
        s = s.replace(',', '').replace('.','') #TBD - replace with punctuation
        s = s[0:280]
        sentences[i] = s
        i += 1
    return sentences

In [60]:

# For Pickle Input
sentences = cleanSentences(sentencesAll)

In [61]:
# Test - 
sentences[99]
# len(sentences)

"a welsh-born english actor in addition to starring roles in big budget hollywood films he has long been heavily involved in films produced by independent producers and art houses bale first caught the public eye when he was cast in the starring role of steven spielberg's empire o"

# Score Sentences in Batches

In [62]:
# score n Sentences at time - to pipeline this
def defineEmbeddings (start, end):    
    embeddings = embed(
        sentences[start:end],
        signature="default",
        as_dict=True)["default"]
    return embeddings

In [65]:
# score sentences
def scoreSentences (embeddings):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        x = sess.run(embeddings)
    return x

In [66]:
# Save Weights - use names based on Movie 
import pickle as pkl

def saveWeights(elmoFN, x):
    with open('./models/'+elmoFN,'wb') as f: pkl.dump(x, f)
    return

## Score N sentences at a time


In [67]:
%matplotlib inline

from IPython.core.magic import register_line_magic


In [68]:
import math 

batch_size = 500                      # Specify number of sentences per file  (was 500)
numPlots = len(sentences)
numfiles = math.ceil(numPlots/batch_size) 

start = 0
end = batch_size
for file in range(1,numfiles+1):
    elmoFN = 'elmoBioweights' + str(start) + 'to' + str(end) + '.pkl'
    print('Score sentences:', elmoFN)

        # Define Embeddings, Score, and Store Weights one Batch of N at time
    embeddings = defineEmbeddings(start, end)
    x = scoreSentences(embeddings)
    saveWeights(elmoFN,x)
    start = end
    end = end+batch_size

Score sentences: elmoBioweights0to500.pkl


## Read and Concatenate all weight arrays in elmoWeights to build


In [73]:
# Read Weights - use names based on actorID 
import pickle as pkl
import numpy as np
import glob

def readElmoWeightFile(file):
    with open(elmodir+file, 'rb') as f:
        elmo_file = pkl.load(f)
    return list(elmo_file)
  
def readWeights(elmodir):
    noFiles = len([name for name in os.listdir(elmodir) ])
    xin = readElmoWeightFile('elmoBioweights0to'+str(batch_size)+'.pkl')
    size = batch_size
    for x in range(1, noFiles):
        start = x*size
        end = start+size
        filename = 'elmoBioweights' + str(start) + 'to' + str(end) + '.pkl'
        print(filename)
        xin += readElmoWeightFile(filename)
    return xin

# Save model and labels catalog.

In [74]:
os.listdir(elmodir)

['elmoBioweights0to500.pkl']

In [75]:

xin = readWeights(elmodir)
print("Elmo Vector Shape {0}, {1}".format( len(xin), len(xin[1])) )

pickle.dump( bio_dict, open( "./models/bio_dict.pkl", "wb" ) )
pickle.dump( xin, open( "./models/elmoBioweights.pkl", "wb" ) )


Elmo Vector Shape 100, 1024


In [76]:
# Load models 

bio_dict = pickle.load( open( './models/bio_dict.pkl', 'rb' ) )
xin = pickle.load( open( './models/elmoBioweights.pkl', 'rb' ) )


In [78]:


topActors = pd.read_csv('./top100Actors.txt', 
                     sep='\t',
                     header=0, 
                    dtype=str)

actorsDF = topActors[['Rank','First Name','Last Name']]
actorsDF['Full Name']= actorsDF['First Name']+' '+actorsDF['Last Name']
actorsDF = actorsDF.drop(['First Name','Last Name'], axis=1)

celebrityLookup={}
for i, row in actorsDF.iterrows():
    celebrityLookup.update({ row['Rank'] : row['Full Name']})

celebrityLookup['10']

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


'Harrison Ford'

## Test Elasticsearch to store the vertor and peform cosine similarity score.

In [80]:
# Load models
pickle.dump( celebrityLookup, open( "../models/celebrityLookup.pkl", "wb" ) )
celebrityLookup = pickle.load( open( '../models/celebrityLookup.pkl', 'rb' ) )


In [36]:
import requests
from pprint import pprint
from datetime import datetime
from elasticsearch import Elasticsearch
import json

es= Elasticsearch([{'host':ESHOST,'port':ESPORT}])

_index='bios'
esIndex = 'http://'+ESHOST+':'+ESPORT+'/'+_index    
_doc= 'bio'

headers={"Content-Type": "application/json"}

In [37]:
request_body = {
  "mappings": {
    "properties": {
      "_actorID" : {
        "type" : "keyword"
      },  
      "celebrity" : {
        "type" : "keyword"
      },         
      "bio_vector": {
        "type": "dense_vector",
        "dims": len(xin[0])
      }

    }
  }
}

try:
    es.indices.delete(index = _index)
except:
    pass
    
es.indices.create(index = _index, body = request_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bios'}

In [ ]:
ids = list(bio_dict.keys())
for i, v in zip(ids, xin):
    _id = str(i)
    #print(_id)
    doc = {
        #_actorID	actor	age	birthdate	gender	height
        "_actorID" : i,
        "celebrity" : celebrityLookup[_id],
        "bio_vector": [float(x) for x in v], 
        }
    r = requests.put(esIndex+'/_doc/'+_id, headers= headers, data = json.dumps(doc))
    print (r.text)

In [ ]:
es.indices.stats(index=_index)

In [ ]:
r = requests.get(esIndex+'/_doc/1')
pprint(r.text)

In [ ]:
vec = json.loads(r.text)["_source"]["bio_vector"]
q={
  "query": {
    "script_score": {
      "query" : {
        "match_all" : {}
      },
      "script": {
        "source": "cosineSimilarity(params.query_vector, doc['bio_vector']) + 1.0", 
        "params": {
          "query_vector": vec
        }
      }
    }
  }
}

res= es.search(index=_index, body=q)

pprint(res)

In [ ]:
[hit['_source']['_actorID'] for hit in res['hits']['hits']]

# REDUCE DIMENSIONS

### Visualize the sentence arrays in reduced Dimensions using PCA and TSNE

Use PCA and t-SNE to reduce the 1,024 dimensions which are output from ELMo down to 2 so that we can review the outputs from the model.

In machine learning, dimensionality reduction is the process of reducing the number of random variables under consideration by obtaining a set of principal variables. By reducing the dimension of your feature space, you have fewer relationships between features to consider which can be explored and visualized easily and also you are less likely to overfit your model. https://www.datacamp.com/community/tutorials/introduction-t-sne

#### Principal Component Analysis or PCA 
Principal Component Analysis (PCA)is a linear feature extraction technique. It performs a linear mapping of the data to a lower-dimensional space in such a way that the variance of the data in the low-dimensional representation is maximized. It does so by calculating the eigenvectors from the covariance matrix. The eigenvectors that correspond to the largest eigenvalues (the principal components) are used to reconstruct a significant fraction of the variance of the original data.

In simpler terms, PCA combines your input features in a specific way that you can drop the least important feature while still retaining the most valuable parts of all of the features. As an added benefit, each of the new features or components created after PCA are all independent of one another.
t-Distributed Stochastic Neighbor Embedding (t-SNE)

#### t-Distributed Stochastic Neighbor Embedding (t-SNE) 
is a non-linear technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. It is extensively applied in image processing, NLP, genomic data and speech processing.


In [ ]:
### NOTE - this takes a bit of time to reduce 1024 by 15316 array into 2 dimensions
from sklearn.decomposition import PCA

pca = PCA(n_components=50)  # was 50
y = pca.fit_transform(xin)

# t-Distributed Stochastic Neighbor Embedding (t-SNE) - Dimensionality Reduction technique
from sklearn.manifold import TSNE
y = TSNE(n_components=2).fit_transform(y)

In [ ]:
# Build a unified data frame (udf) with columns: MovieID, Plot, Genre, and Y-transform
# udf = pd.DataFrame(columns=['MovieID', 'Plot', 'Genre','Y-Transform'])

i=0
row_list = []
for key in plot_dict: 
    row_dict = {'MovieID':key,
                'Plot':sentences[i],
                'Genre':genre_dict[key],
                'Y-Transform-X': y[i][0],
                'Y-Transform-Y': y[i][1],
               }
    row_list.append(row_dict)
    i += 1
udf = pd.DataFrame(row_list)

In [ ]:
udf.to_csv("UnifiedData.csv", index=False)

In [ ]:
# Test below

In [ ]:
udf.head()

In [ ]:
udf = pd.read_csv("../data/elmoWeightsAll/UnifiedData.csv", dtype={'MovieID':str} )
udf.head()